# Advanced Machine Learning - Project 1

## TODO
- renaming vars
- check for multiple nans (and possibly remove?)
- nan imputation 
- check how to pre-process features (How to encode categorical data?, How to scale continuous?)
- read into hybrid Bayesian Networks? How to do?
- train test split, do KFold?

## Learning approach
- (maybe feature selection)
1. learn structure
2. learn CPDs

## Make predictions

## Interpret

In [1]:
# Imports

# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

# Data Upscaling or Downscaling
from sklearn.model_selection import train_test_split
import imblearn.over_sampling as over_sampling
import imblearn.under_sampling as under_sampling

# Bayesian network and estimators
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

from pgmpy.estimators import ParameterEstimator
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import BayesianEstimator

from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.estimators import ExhaustiveSearch
from pgmpy.estimators import HillClimbSearch




d:\OneDrive - Universidade de Coimbra\Mestrado\Files\1Semestre\AAA\P\P1\venv_AML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Reading the dataset
df = pd.read_csv("../CHD_dataset.csv")

df.head(10)

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,male,39,4.0,No,0.0,0.0,0,0,No,195.0,106.0,70.0,26.97,80.0,77.0,0
1,female,46,2.0,No,0.0,0.0,0,0,No,250.0,121.0,81.0,28.73,95.0,76.0,0
2,male,48,1.0,Yes,20.0,0.0,0,0,No,245.0,127.5,80.0,25.34,75.0,70.0,0
3,female,61,3.0,Yes,30.0,0.0,0,1,No,225.0,150.0,95.0,28.58,65.0,103.0,1
4,female,46,3.0,Yes,23.0,0.0,0,0,No,285.0,130.0,84.0,23.10,85.0,85.0,0
5,female,43,2.0,No,0.0,0.0,0,1,No,228.0,180.0,110.0,30.30,77.0,99.0,0
6,female,63,1.0,No,0.0,0.0,0,0,No,205.0,138.0,71.0,33.11,60.0,85.0,1
7,female,45,2.0,Yes,20.0,0.0,0,0,No,313.0,100.0,71.0,21.68,79.0,78.0,0
8,male,52,1.0,No,0.0,0.0,0,1,No,260.0,141.5,89.0,26.36,76.0,79.0,0
9,male,43,1.0,Yes,30.0,0.0,0,1,No,225.0,162.0,107.0,23.61,93.0,88.0,0


In [3]:
# Checking for NaN
print(f'NaN in the dataset:\n {df.isnull().sum()}')

# Checking for duplicates
print( f'\nDuplicated: {Counter(df.duplicated())}')



NaN in the dataset:
 Sex                  0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

Duplicated: Counter({False: 4240})


In [10]:
print( df["BPMeds"].min())
print( df["BPMeds"].max())
print( df["BPMeds"].mean())

print( df["totChol"].min())
print( df["totChol"].max())
print( df["totChol"].mean())

print( df["glucose"].min())
print( df["glucose"].max())
print( df["glucose"].mean())
print( df["glucose"].std())

0.0
1.0
0.029615476474802963
107.0
696.0
236.69952267303103
40.0
394.0
81.96365524402907
23.954334811344744


In [ ]:
# Choose what to do with NaN's

# Get the mean of cigs per day of smokers
mean_cigs_per_day = df[ (df["currentSmoker"] == "Yes") & ( df["cigsPerDay"] > 0 ) ]["cigsPerDay"].mean(skipna=True)

# Fill the NaN values of smokers with the mean, of non smokers with 0
df.loc[ (df["cigsPerDay"].isna()) & ( df["currentSmoker"] == "Yes") , "cigsPerDay" ] =  mean_cigs_per_day
df.loc[ (df["cigsPerDay"].isna()) & ( df["currentSmoker"] == "No"), "cigsPerDay" ] = 0

# Print just to check if operation above was completed correctly
# print(f'NaN in the dataset:\n {df.isnull().sum()["cigsPerDay"]}')

"""
TODO: Fill education, BPMeds, totChol, BMI, heartRate and glucose
      Can we generalize filling the NaN with mean in a function, inputing the df and the column?????
      Maybe introduce some stocastic number to the mean, based on the standard deviation??? just to
      introduce some variability in the created values
"""
# Get mean BMI for male and female
mean_male = df.loc[ df["Sex"] == "male", "BMI"].mean()
mean_female = df.loc[ df["Sex"] == "female", "BMI"].mean()

# Fill NaN values
df.loc[ ( df["Sex"] == "male" ) & ( df["BMI"].isna() ), "BMI"] = mean_male
df.loc[ ( df["Sex"] == "female" ) & ( df["BMI"].isna() ), "BMI"] = mean_female

In [ ]:
# Removing target variable from the dataset
X = df.drop("TenYeahCHD", axis=1)
y = df["TenYearCHD"]